# Suspicious Calculator Usage
Detects suspicious use of calc.exe with command line parameters or in a suspicious directory, which is likely caused by some PoC or detection evasion

## Rule Content
```
- title: Suspicious Calculator Usage
  id: 737e618a-a410-49b5-bec3-9e55ff7fbc15
  description: Detects suspicious use of calc.exe with command line parameters or
    in a suspicious directory, which is likely caused by some PoC or detection evasion
  status: experimental
  references:
  - https://twitter.com/ItsReallyNick/status/1094080242686312448
  author: Florian Roth
  date: 2019/02/09
  tags:
  - attack.defense_evasion
  - attack.t1036
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine: '*\calc.exe *'
    selection2:
      Image: '*\calc.exe'
    filter2:
      Image: '*\Windows\Sys*'
    condition: selection1 or ( selection2 and not filter2 )
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*\\calc.exe\ * OR (process_path.keyword:*\\calc.exe AND (NOT (process_path.keyword:*\\Windows\\Sys*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()